In [29]:
!vimr 2024-12.txt

In [58]:
from collections import defaultdict

data = """RRRRIICCFF
RRRRIICCCF
VVRRRCCFFF
VVRCCCJFFF
VVVVCJJCFE
VVIVCCJJEE
VVIIICJJEE
MIIIIIJJEE
MIIISIJEEE
MMMISSJEEE"""

from pathlib import Path
#data = Path('2024-12.txt').read_text()
def make_grid(data):
    return {(x,y):r for y,l in enumerate(data.splitlines()) for x,r in enumerate(l) }

grid = make_grid(data)

In [45]:
grid_n = dict()

for (y,x),r in grid.items():
    neighbors = [n for dx,dy in ((-1,0),(1,0),(0,1),(0,-1)) if (n:=grid.get((y+dy, x+dx), None)) == r]
    fences = 4 - len(neighbors)
    grid_n[(y,x)] = (r, (y,x), fences)

In [87]:


def make_corner_grid(grid):

    grid_c = dict()
    
    for (y,x),r in grid.items():
        corners = 0
        #print(x,y,r)
        for dy,dx in ((-1,-1),(-1,1),(1,1),(1,-1)):
            a,b,c = grid.get((y-dy,x), None), grid.get((y-dy,x-dx), None), grid.get((y,x-dx), None)
            if r == a == b == c:   # AB
                pass               # RC
                #print('inside', r, a, b,c)
            elif r != a and r != c:
                assert r!=c # no corner -corner regions
                corners +=1
                #print('outer corner', r, a, b,c)
            elif r != a or r != c:  # a.      A.
                pass                 # RC  or  Rc
                #print('strait',r, a, b,c)
            elif r == a == c and  r != b:   # Ab
                corners += 1                # RC  inner corner
                #print('inner corner', r, a, b,c)
            else:
                print('---')
                print(a,b)
                print(r.lower(),c)
        
        grid_c[(y,x)] = (r, (y,x), corners)
    return grid_c
grid_c = make_corner_grid(grid)

In [88]:
grid

{(0, 0): 'R',
 (1, 0): 'R',
 (2, 0): 'R',
 (3, 0): 'R',
 (4, 0): 'I',
 (5, 0): 'I',
 (6, 0): 'C',
 (7, 0): 'C',
 (8, 0): 'F',
 (9, 0): 'F',
 (0, 1): 'R',
 (1, 1): 'R',
 (2, 1): 'R',
 (3, 1): 'R',
 (4, 1): 'I',
 (5, 1): 'I',
 (6, 1): 'C',
 (7, 1): 'C',
 (8, 1): 'C',
 (9, 1): 'F',
 (0, 2): 'V',
 (1, 2): 'V',
 (2, 2): 'R',
 (3, 2): 'R',
 (4, 2): 'R',
 (5, 2): 'C',
 (6, 2): 'C',
 (7, 2): 'F',
 (8, 2): 'F',
 (9, 2): 'F',
 (0, 3): 'V',
 (1, 3): 'V',
 (2, 3): 'R',
 (3, 3): 'C',
 (4, 3): 'C',
 (5, 3): 'C',
 (6, 3): 'J',
 (7, 3): 'F',
 (8, 3): 'F',
 (9, 3): 'F',
 (0, 4): 'V',
 (1, 4): 'V',
 (2, 4): 'V',
 (3, 4): 'V',
 (4, 4): 'C',
 (5, 4): 'J',
 (6, 4): 'J',
 (7, 4): 'C',
 (8, 4): 'F',
 (9, 4): 'E',
 (0, 5): 'V',
 (1, 5): 'V',
 (2, 5): 'I',
 (3, 5): 'V',
 (4, 5): 'C',
 (5, 5): 'C',
 (6, 5): 'J',
 (7, 5): 'J',
 (8, 5): 'E',
 (9, 5): 'E',
 (0, 6): 'V',
 (1, 6): 'V',
 (2, 6): 'I',
 (3, 6): 'I',
 (4, 6): 'I',
 (5, 6): 'C',
 (6, 6): 'J',
 (7, 6): 'J',
 (8, 6): 'E',
 (9, 6): 'E',
 (0, 7): 'M',
 (1, 7

In [89]:
def pick_region(start, grid_n):
    open = {start}
    close = set()
    while open:
        node = open.pop()
        r,(y,x), fences = node
        neighbors = {n for dx,dy in ((-1,0),(1,0),(0,1),(0,-1)) if (n:=grid_n.get((y+dy, x+dx), [None]))[0] == r}
        neighbors = neighbors - close
        open.update(neighbors)
        close.add(node)
    return close


def price(grid_n):
    regions = []
    from copy import copy
    source = set(grid_n.values())
    while source:
        reg = pick_region(source.pop(), grid_n)
        source = source - reg
        regions.append(reg)
    l = [(next(iter(r))[0], len(r), sum(c[2] for c in r)) for r in regions]
    l.sort()
    return sum([a*p for _,a,p in l])

price(grid_n) # 1431316

1930

In [90]:
price(make_corner_grid(make_grid(
"""AAAAAA
   AAABBA
   AAABBA
   ABBAAA
   ABBAAA
   AAAAAA""".replace(' ','')))) # 368

368

In [92]:
price(make_corner_grid(make_grid(
"""EEEEE
EXXXX
EEEEE
EXXXX
EEEEE""".replace(' ','')))) # 236

236

In [93]:
price(make_corner_grid(make_grid(
"""OOOOO
OXOXO
OOOOO
OXOXO
OOOOO""".replace(' ',''))))

436

In [94]:



price(make_corner_grid(make_grid(Path('2024-12.txt').read_text())))

821428